In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import os 
from glob import glob 

# 데이터 로드 

In [3]:
!unzip /content/drive/MyDrive/별떠라가자_공유폴더/분석데이터/뉴데이터_0414/T아카데미-모델링.zip

Archive:  /content/drive/MyDrive/별떠라가자_공유폴더/분석데이터/뉴데이터_0414/T아카데미-모델링.zip
   creating: T아카데미-모델링/
  inflating: T아카데미-모델링/A77E8CE3.csv  
  inflating: T아카데미-모델링/.DS_Store  
  inflating: __MACOSX/T아카데미-모델링/._.DS_Store  
  inflating: T아카데미-모델링/F0A475A1.csv  
  inflating: T아카데미-모델링/88826FAB.csv  
  inflating: T아카데미-모델링/DB7B0571.csv  
  inflating: T아카데미-모델링/912BB33B.csv  
  inflating: T아카데미-모델링/4E34891D.csv  
  inflating: T아카데미-모델링/B95FBB89.csv  
  inflating: T아카데미-모델링/D788034C.csv  
  inflating: T아카데미-모델링/1DBEBE62.csv  
  inflating: __MACOSX/T아카데미-모델링/._1DBEBE62.csv  
  inflating: T아카데미-모델링/483AA738.csv  


In [4]:
!cp /content/drive/MyDrive/별떠라가자_공유폴더/분석데이터/기존100hz데이터/20211203.csv /content/T아카데미-모델링
!cp /content/drive/MyDrive/별떠라가자_공유폴더/분석데이터/기존100hz데이터/20211204.csv /content/T아카데미-모델링
!cp /content/drive/MyDrive/별떠라가자_공유폴더/분석데이터/기존100hz데이터/20211205.csv /content/T아카데미-모델링

In [5]:
temp_c = ['DATE', 'ID', 'ACCEL_X', 'ACCEL_Y', 'ACCEL_Z', 'GYRO_X', 'GYRO_Y',
       'GYRO_Z', 'PITCH', 'ROLL', 'LAT', 'LON', 'VELOCITY', 'ALT', 'EVENT0',
       'EVENT1', 'EVENT2', 'EVENT3']

In [6]:
path = '/content/T아카데미-모델링'
csv_dir = os.listdir('/content/T아카데미-모델링')
df_dict = {} 

for i in range(len(csv_dir)):
  name = csv_dir[i].split('.')[0]
  if name in ['20211203','20211204','20211205']:
    df_path = os.path.join(path,csv_dir[i])
    temp_df = pd.read_csv(df_path,header=None)
    temp_df.columns = temp_c
    df_dict[name] = temp_df
  else:
    try:
      df_path = os.path.join(path,csv_dir[i])
      df_dict[name] = pd.read_csv(df_path)
    except:
      print(name)
id_list = list(df_dict.keys())

# 데이터 전처리 

### 0값 확인 

가속도, 자이로 x,y,z가 모두 0인 경우 제거 

In [7]:
def remove_0(df):
  a = df.loc[(df['ACCEL_X']==0) & (df['ACCEL_Y']==0) & (df['ACCEL_Z']==0)].index
  df = df.drop(a).reset_index().drop(columns='index')
  a = df.loc[(df['GYRO_X']==0) & (df['GYRO_Y']==0) & (df['GYRO_Z']==0)].index
  df = df.drop(a).reset_index().drop(columns='index')
  return df 

In [8]:
id_list = list(df_dict.keys())
id = id_list[0]
for id in id_list:
  df = df_dict[id]
  df = remove_0(df) # 0값 제거 == 결측치 제거 
  #컬럼 선별 
  df = df[['DATE','ID','ACCEL_X','ACCEL_Y','ACCEL_Z','GYRO_X','GYRO_Y','GYRO_Z']]
  df_dict[id] = df 

## 스케일링 
- minmax로 진행 
- 모든 csv 중 min, max 값으로 진행 

In [9]:
id = id_list[0]

min_list = []
max_list = [] 
for id in id_list:
  df = df_dict[id]
  df  = df.drop(columns=['DATE','ID'])
  min = np.min(df).values
  max = np.max(df).values
  min_list.append(min)
  max_list.append(max)
min_list = np.min(min_list,axis=0)
max_list = np.max(max_list,axis=0)

## 타임라인 선별 
- 20개보다 작은 경우는 모두 제거 

In [10]:
def make_timestamps(temp_df):
  temp_timestamps = np.array(temp_df['DATE'].value_counts()[temp_df['DATE'].value_counts()>=20].reset_index()['index'])
  return temp_timestamps

def make_drop_value(time_df):
  time_index = time_df.index
  time_length = len(time_index)
  drop_length = time_length - 20 
  drop_index = np.random.choice(time_index,drop_length,replace=False)
  drop_value = np.array(time_df.drop(drop_index).drop(columns = ['DATE','ID']))
  return drop_value 
  
def make_scaling(drop_value):
  global min_list,max_list
  drop_value = (drop_value - min_list)/(max_list-min_list)
  drop_value = drop_value.astype(np.float16)
  return drop_value

In [ ]:
preprocessed_value = [] 
for id in id_list[:1]:
  df = df_dict[id]
  timestamps = make_timestamps(df)
  for timestamp in tqdm(timestamps):
    time_df = df[df['DATE']==timestamp]
    drop_value = make_drop_value(time_df)
    # drop_value = make_scaling(drop_value)
    preprocessed_value.append(drop_value)
  

 39%|███▉      | 13084/33409 [45:02<1:09:37,  4.87it/s]

In [ ]:
(1000000,20,6)

In [ ]:
drop_value = make_drop_value(time_df)

In [ ]:
drop_value.shape

(20, 6)

In [ ]:
np.array(preprocessed_value)[6]

array([[0.4458, 0.4233, 0.7163, 0.5493, 0.536 , 0.3586],
       [0.4458, 0.4233, 0.7163, 0.5493, 0.536 , 0.3586],
       [0.4458, 0.4233, 0.7163, 0.5493, 0.536 , 0.3586],
       [0.4458, 0.4233, 0.7163, 0.5493, 0.536 , 0.3586],
       [0.4338, 0.4597, 0.509 , 0.499 , 0.532 , 0.3582],
       [0.4338, 0.4597, 0.509 , 0.499 , 0.532 , 0.3582],
       [0.4617, 0.3977, 0.75  , 0.5063, 0.532 , 0.3574],
       [0.4617, 0.3977, 0.75  , 0.5063, 0.532 , 0.3574],
       [0.4302, 0.317 , 0.8647, 0.5303, 0.534 , 0.3577],
       [0.4302, 0.317 , 0.8647, 0.5303, 0.534 , 0.3577],
       [0.4282, 0.582 , 0.8604, 0.5845, 0.538 , 0.3591],
       [0.4282, 0.582 , 0.8604, 0.5845, 0.538 , 0.3591],
       [0.4265, 0.4976, 0.699 , 0.556 , 0.5347, 0.3584],
       [0.44  , 0.4639, 0.834 , 0.51  , 0.5317, 0.3557],
       [0.44  , 0.4639, 0.834 , 0.51  , 0.5317, 0.3557],
       [0.4219, 0.4343, 0.7744, 0.515 , 0.531 , 0.358 ],
       [0.437 , 0.468 , 0.683 , 0.5317, 0.533 , 0.3586],
       [0.437 , 0.468 , 0.683 ,